In [147]:
import tensorflow as tf
import numpy as np
import pandas as pd
import itertools

In [167]:
df=pd.read_csv("Camera.csv",delimiter=";")
df.columns = [c.replace(' ', '_') for c in df.columns]
df.columns = [c.replace('.', '').replace('.','').replace('(','').replace(')','') for c in df.columns]
df=df[df.columns.difference(["Model",])]

In [168]:
Converted_Df =pd.DataFrame(np.nan_to_num(df[2:].values.astype(float)))

In [169]:
Converted_Df.columns=df.columns

In [170]:
train=Converted_Df.sample(frac=0.8,random_state=200)
test=Converted_Df.drop(train.index)
train.columns

Index(['Dimensions', 'Effective_pixels', 'Low_resolution', 'Macro_focus_range',
       'Max_resolution', 'Normal_focus_range', 'Price', 'Release_date',
       'Storage_included', 'Weight_inc_batteries', 'Zoom_tele_T',
       'Zoom_wide_W'],
      dtype='object')

In [171]:
FEATURES=Converted_Df.columns.values.tolist()

In [172]:
feature_cols = [tf.feature_column.numeric_column(k,dtype=tf.float32)  for k in FEATURES]
feature_cols

[_NumericColumn(key='Dimensions', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='Effective_pixels', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='Low_resolution', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='Macro_focus_range', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='Max_resolution', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='Normal_focus_range', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='Price', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='Release_date', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='Storage_included', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='We

In [192]:
regressor = tf.contrib.learn.DNNRegressor(hidden_units=[80, 80],feature_columns=feature_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000000011901080>, '_master': '', '_tf_random_seed': None, '_environment': 'local', '_save_checkpoints_steps': None, '_task_type': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_model_dir': 'C:\\Users\\bkaushal\\AppData\\Local\\Temp\\tmp0e4z01tm', '_task_id': 0, '_save_checkpoints_secs': 600, '_num_worker_replicas': 0, '_is_chief': True}


In [193]:
def get_input_fn(train, num_epochs=None, shuffle=True):
  return tf.estimator.inputs.pandas_input_fn(
      x=train,
      y = train.Price,
      num_epochs=num_epochs,
      shuffle=shuffle)

In [194]:
regressor.fit(input_fn=get_input_fn(train),steps=5000,)



Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\bkaushal\AppData\Local\Temp\tmp0e4z01tm\model.ckpt.
INFO:tensorflow:loss = 363570.0, step = 1
INFO:tensorflow:global_step/sec: 314.591
INFO:tensorflow:loss = 3148.51, step = 101 (0.321 sec)
INFO:tensorflow:global_step/sec: 277.889
INFO:tensorflow:loss = 751.068, step = 201 (0.360 sec)
INFO:tensorflow:global_step/sec: 265.358
INFO:tensorflow:loss = 303.609, step = 301 (0.376 sec)
INFO:tensorflow:global_step/sec: 263.263
INFO:tensorflow:loss = 786.538, step = 401 (0.380 sec)
INFO:tensorflow:global_step/sec: 277.889
INFO:tensorflow:loss = 197.788, step = 501 (0.359 sec

DNNRegressor(params={'optimizer': None, 'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x0000000011545828>, 'feature_columns': (_NumericColumn(key='Dimensions', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='Effective_pixels', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='Low_resolution', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='Macro_focus_range', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='Max_resolution', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='Normal_focus_range', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='Price', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='Release_date', shape=(1,), default_value=None, dtype=tf.fl

In [195]:
ev = regressor.evaluate(
    input_fn=get_input_fn(test, num_epochs=2, shuffle=False))

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-08-23-13:23:19
INFO:tensorflow:Restoring parameters from C:\Users\bkaushal\AppData\Local\Temp\tmp0e4z01tm\model.ckpt-5000
INFO:tensorflow:Finished evaluation at 2017-08-23-13:23:19
INFO:tensorflow:Saving dict for global step 5000: global_step = 5000, loss = 238.708


In [196]:
loss_score = ev["loss"]
print("Loss: {0:f}".format(loss_score))

predict_df=pd.DataFrame([2002,1600.0,800.0,1.0,38.0,38.0,40.0,20.0,8.0,180.0,86.0,0.0]).T
predict_df.columns=df.columns


Loss: 238.707855


In [197]:
y = regressor.predict(
    input_fn=get_input_fn(predict_df, num_epochs=1, shuffle=False))

Instructions for updating:
Please switch to predict_scores, or set `outputs` argument.
INFO:tensorflow:Restoring parameters from C:\Users\bkaushal\AppData\Local\Temp\tmp0e4z01tm\model.ckpt-5000


In [198]:
predictions = list(y)


In [199]:
predictions

[-313.49844]